In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
Data = pd.read_csv('../data/business_consulting_data.csv')

In [ ]:
Data

In [ ]:
Data_F = Data.copy()

In [ ]:
Data_F

In [ ]:
import pandas as pd
import re
import html

# Define your question patterns mapped to clean column names
questions = {
    "Have you used business consulting services before\\?": "Have you used business consulting services before?",
    "How long has the business been running\\?": "How long has the business been running?",
    "What is the business's annual turnover/sales\\?": "What is the business annual turnover/sales?",
    "How many employees do you have\\?": "How many employees do you have?",
    "What industry is your business in\\?": "What industry is your business in?",
    "Which type\\(s\\) of consulting are you interested in\\?": "Which type(s) of consulting are you interested in?",
    "What are your goals for this service\\?": "What are your goals for this service?",
    "How long do you need a consultant\\?": "How long do you need a consultant?",
    "How would you like to work with the consultant\\?": "How would you like to work with the consultant?",

}

# Function to extract answers
def extract_answers(row):
    extracted = {v: None for v in questions.values()}  # initialize all columns with None
    if not row:
        return pd.Series(extracted)

    # Decode HTML characters (like &#039; -> ')
    row = html.unescape(row)

    # Clean unwanted encoding artifacts (like Â, â€™)
    row = re.sub(r'[^\x00-\x7F]+', ' ', row)

    # Split based on questions
    pattern = "(" + "|".join(questions.keys()) + ")"
    parts = re.split(pattern, row)

    i = 1
    while i < len(parts) - 1:
        q = parts[i].strip() if parts[i] else ''
        a = parts[i + 1].strip() if parts[i + 1] else ''
        for k, v in questions.items():
            if re.fullmatch(k, q):
                # If 'Additional details' already exists and new text comes, append
                if v == "Additional details" and extracted[v]:
                    extracted[v] += " " + a
                else:
                    extracted[v] = a
        i += 2

    return pd.Series(extracted)

# Apply the function
result_df = Data['Que/Ans'].apply(extract_answers)

# Final cleaning (optional, just in case)
result_df = result_df.replace({r'[^\x00-\x7F]+': ''}, regex=True)

# Preview
result_df.head()

# Optional save
# result_df.to_csv('cleaned_output.csv', index=False)
# result_df.to_excel('cleaned_output.xlsx', index=False)


In [ ]:
Data = pd.concat([Data, result_df], axis=1)

In [ ]:
Data

In [ ]:
Datak = Data
columns_to_drop = ['S.no', 'Response','Country','Additional Details',
    'Initial', 'Name','Time', 'Badge 1','Additional','Credits',
    'Badge 2', 'Badge 3', 'Badge 4', 'Category', 'Que/Ans'
]

# Drop the columns (if they exist in the DataFrame)
Datak = Datak.drop(columns=[col for col in columns_to_drop if col in Datak.columns])

Datak.to_csv("../data/sample_data_for_prediction_business_consulting.csv", index=False)



In [ ]:
columns_to_drop = ['S.no', 'Lead Id', 'Response','Country','Additional Details',
    'Initial', 'Name','Time', 'Badge 1','Additional',
    'Badge 2', 'Badge 3', 'Badge 4', 'Category', 'Que/Ans'
]

# Drop the columns (if they exist in the DataFrame)
Data = Data.drop(columns=[col for col in columns_to_drop if col in Data.columns])

In [ ]:
Data.columns

In [ ]:
# Load dataset
df = Data

# Define target variable
target = "Credits"

# Define features
features = [
   'Location', 'Urgent', 'High', 'Verified', 'Frequent',
       'Have you used business consulting services before?',
       'How long has the business been running?',
       'What is the business annual turnover/sales?',
       'How many employees do you have?', 'What industry is your business in?',
       'Which type(s) of consulting are you interested in?',
       'What are your goals for this service?',
       'How long do you need a consultant?',
       'How would you like to work with the consultant?']


In [ ]:
# Drop rows with missing target values
df = df.dropna(subset=[target])

In [ ]:

# Identify categorical and numerical features
cat_features = df[features].select_dtypes(include=['object']).columns.tolist()
num_features = df[features].select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:

# Fill missing values
df[cat_features] = df[cat_features].fillna("Unknown")
df[num_features] = df[num_features].fillna(df[num_features].median())

In [ ]:
cat_features

In [ ]:
num_features

In [ ]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Convert to CatBoost Pool format
train_pool = Pool(X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(X_test, label=y_test, cat_features=cat_features)


In [ ]:

# Initialize CatBoost model
model = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    loss_function='RMSE',
    cat_features=cat_features,
    verbose=100
)

In [ ]:
# Train model
model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

In [ ]:

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

In [ ]:
validated = df[features]

In [ ]:
# Make predictions on NewData without specifying cat_features
NewDataPrediction = model.predict(df[features])

In [ ]:
import catboost
import numpy as np
import pandas as pd

# Get SHAP values, including the base value, for all records in NewData
shap_values = model.get_feature_importance(
    data=catboost.Pool(validated, cat_features=cat_features),
    type="ShapValues"
)

# Get predicted values
predicted_values = model.predict(validated)

# Convert SHAP values to DataFrame (include base value)
shap_columns = list(validated.columns) + ["base_value"]
shap_df = pd.DataFrame(shap_values, columns=shap_columns)

# Add predicted values column
shap_df["predicted_value"] = predicted_values

# Display SHAP values with predictions
shap_df  # Show SHAP values for the first few records including prediction




In [ ]:
shap_df1 = shap_df.merge(Data_F[["Credits","Lead Id"]], left_index=True, right_index=True)

In [ ]:
Datak

In [ ]:
shap_df1

In [ ]:
shap_df1.to_csv("../data/New_Scores_Business_Consulting.csv", index=False)

In [ ]:
# Save model
joblib.dump(model, "../model/business_consulting.pkl")

print("Model saved successfully.")

In [ ]:
# Save the filtered dataset
#df_results.to_csv("Scores_Actual_vs_Predicted.csv", index=False)